## Query with Memory from txt File 

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [2]:
from langchain.document_loaders import TextLoader
loader = TextLoader("test_data.txt")
documents = loader.load()

In [3]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000


In [4]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [5]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [6]:
query = "From whom is the Document?"
result = qa({"question": query})

In [7]:
result["answer"]

' The document is from Linea Schmidt.'

In [8]:
query2 = "Is there any information about her profession?"
result2 = qa({"question": query2})

In [9]:
result2["answer"]

" I don't know."

## Query with Memory from Web

In [10]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [18]:
from langchain.memory import ConversationSummaryMemory
llm = OpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [21]:
qa("How do agents use Task decomposition?")

{'question': 'How do agents use Task decomposition?',
 'chat_history': [SystemMessage(content='')],
 'answer': 'Agents utilize task decomposition by breaking down large tasks into smaller, manageable subgoals. This allows them to efficiently handle complex tasks. Task decomposition can be achieved through different methods such as using simple prompting, task-specific instructions, or human inputs. The agent can then plan ahead and determine the necessary steps to achieve each subgoal. This approach enables the agent to tackle complicated tasks in a more organized and systematic manner.'}

In [22]:
qa("What are the various ways to implement memory to support it?")

{'question': 'What are the various ways to implement memory to support it?',
 'chat_history': [SystemMessage(content='\nThe human asks how agents use Task decomposition. The AI explains that agents utilize task decomposition by breaking down large tasks into smaller, manageable subgoals, which can be achieved through different methods such as using simple prompting, task-specific instructions, or human inputs. This allows the agent to plan ahead and determine the necessary steps to achieve each subgoal, enabling them to tackle complicated tasks in a more organized and systematic manner.')],
 'answer': "The different methods to implement memory in order to support task decomposition are:\n\n1. Long Term Memory Management: This involves storing and organizing information in a long-term memory system. It allows the agent to retain knowledge of past tasks, subgoals, and their corresponding solutions. This memory can be accessed during task decomposition to retrieve relevant information and

## Query from Neo4J database 

In [24]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self._driver = GraphDatabase.driver(uri, auth=(user, pwd))

    def close(self):
        self._driver.close()

In [34]:
class Neo4jLoader:
    def __init__(self, connection, query):
        self.connection = connection
        self.query = query

    def load(self):
        with self.connection._driver.session() as session:
            results = session.run(self.query)
            data = [record for record in results]
        return data

In [37]:
class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def fetch_data(self):
        with self._driver.session() as session:
            return session.read_transaction(self._fetch_data_from_db)

    @staticmethod
    def _fetch_data_from_db(tx):
        query = """
        MATCH (n:Study)  // Assuming you have a label 'Study' for your nodes
        RETURN n.description  // Assuming each study node has a 'description' property
        """
        result = tx.run(query)
        return [record["n.description"] for record in result]

In [39]:
# Use your Neo4j credentials here
neo_service = Neo4jService(uri="bolt://localhost:7687", user="neo4j", password="password")
data = neo_service.fetch_data()
neo_service.close()

/var/folders/j0/gd9db9l10k5grhsvfgxwyll00000gn/T/ipykernel_38522/4294405743.py:11: DeprecationWarning: read_transaction has been renamed to execute_read
  return session.read_transaction(self._fetch_data_from_db)


In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

response = qa("How do agents use Task decomposition?")
print(response)

AttributeError: 'NoneType' object has no attribute 'page_content'